In [1]:
%load_ext rustdef

load rustdef


In [2]:
%rustdef depends ndarray

In [3]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    println!("{}", a);
    Ok(z.into())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at python3.7
   Compiling rustdef v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef)
   Compiling rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e)
    Finished release [optimized] target(s) in 4.05s==================> ] 79/80
📦 Built wheel for CPython 3.7m to /Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/target/wheels/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl


In [4]:
import numpy as np

In [5]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

2.4


array([ 4.7 , 18.58,  4.96])

In [6]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    Ok(z.into())
}

// mutable example
#[pyfunction]
fn mult_inline<'a>(a: f64, x: Array<'a, f64>) -> PyResult<()> {
    let mut x = x.as_mut_array()?;
    x *= a;
    Ok(())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at python3.7
   Compiling rustdef_cell_9aaabe0b68cfd1aea3f69d328ce1b5efab1f3839 v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef_cell_9aaabe0b68cfd1aea3f69d328ce1b5efab1f3839)
    Finished release [optimized] target(s) in 2.61s==================> ] 79/80
📦 Built wheel for CPython 3.7m to /Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/target/wheels/rustdef_cell_9aaabe0b68cfd1aea3f69d328ce1b5efab1f3839-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl


In [7]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.17845857, 0.86535655, 0.26647273, 0.09533224, 0.54860756,
        0.18959229, 0.67551889, 0.08966322, 0.69370085, 0.79113257]),
 array([0.07645075, 0.5222156 , 0.5302601 , 0.00812685, 0.5225054 ,
        0.42701628, 0.84920635, 0.99116843, 0.78219441, 0.25152478]))

In [8]:
axpy(3.0, x, y)

array([0.61182648, 3.11828526, 1.32967828, 0.29412356, 2.16832808,
       0.99579314, 2.87576302, 1.2601581 , 2.86329696, 2.62492248])

In [9]:
mult_inline(12.0, x)

In [10]:
x

array([ 2.1415029 , 10.38427865,  3.19767273,  1.14398686,  6.58329073,
        2.27510744,  8.10622669,  1.07595865,  8.32441019,  9.49359079])